In [28]:
import pandas as pd
import cobra
import json

In [29]:
def save_dict(data, name):
    with open(name, 'w' ) as file:
        json.dump( data, file )

# Creating COMPM media

We compute the competition inducing media by using the minimal minimum flux of all common exchange reactions. First we load all required stuff.

In [3]:
model_DP = cobra.io.read_sbml_model("models/consistent_DP_SNM.xml")
model_SA = cobra.io.read_sbml_model("models/consistent_iYS854_SNM.xml")
print("Growth: ", model_DP.slim_optimize())
print("Growth: ", model_SA.slim_optimize())

Growth:  0.2798991966652624
Growth:  2.5586946126133867


In [4]:
snm3 = pd.read_csv("SNM3.csv", sep =";")
snm3.head()

,Compound,BiGG,ModelSeed,KEGG
0,Alanine,ala__L,cpd00035,C00041
1,Arginine,arg__L,cpd00051,C00062
2,Cysteine,cys__L,cpd00084,C00097
3,Glutamic acid,glu__L,cpd00023,C00025
4,Glycine,gly,cpd00033,C00037


In [5]:
# Union of exchange reactions
ex_DP_ids = set(map(lambda x:x.id, model_DP.exchanges))
ex_SA_ids = set(map(lambda x:x.id, model_SA.exchanges))
ex_union_ids = ex_DP_ids.union(ex_SA_ids)
print("Number of united exchange reactions: ", len(ex_union_ids))

Number of united exchange reactions:  111


In [6]:
fva_DP = cobra.flux_analysis.flux_variability_analysis(model_DP)
fva_SA = cobra.flux_analysis.flux_variability_analysis(model_SA)

## 1) Computing COMPM media
We compute the compm media, as following:
* If the exchange reaction is common between the species, we take the minimum flux.
* If the exchange reaction is not common, we take the minFVA value of the specific species 

Note that we make **two speperate media**, for each species one, as a **medium can only contain exchange reactions that are present in the model**.

In [7]:
def compute_COMPM(fva_DP, fva_SA):
    media_COMPM_DP = dict()
    media_COMPM_SA = dict()
    for ex in ex_union_ids:
        if ex in fva_DP.index and ex in fva_SA.index:
            # Take minimum of both minima if common
            flux = min(fva_DP.loc[ex].minimum, fva_SA.loc[ex].minimum)
            if flux < 0:
                    media_COMPM_DP[ex] = abs(flux)
                    media_COMPM_SA[ex] = abs(flux)
        elif ex in fva_DP.index:
            # Take just the minimum if not common
            flux = fva_DP.loc[ex].minimum
            if flux < 0:
                    media_COMPM_DP[ex] = abs(flux)
        elif ex in fva_SA.index:
            # Take just the minimum if not common
            flux = fva_SA.loc[ex].minimum
            if flux < 0:
                    media_COMPM_SA[ex] = abs(flux)
        else:
            print("error")
    return media_COMPM_DP, media_COMPM_SA       

In [8]:
media_COMPM_DP, media_COMPM_SA = compute_COMPM(fva_DP,fva_SA)

In [9]:
print("Length of media: ", len(media_COMPM_DP))
print(media_COMPM_DP)
save_dict(media_COMPM_DP, "compm_DP.json")

Length of media:  36
{'EX_cobalt2_e': 0.009894648965030426, 'EX_thr__L_e': 0.14751502550566087, 'EX_o2_e': 20.0, 'EX_so4_e': 0.010557173971642781, 'EX_zn2_e': 0.008915031973978057, 'EX_gly_e': 10.0, 'EX_nac_e': 7.953461359822521, 'EX_fum_e': 10.0, 'EX_arg__L_e': 10.0, 'EX_trp__L_e': 0.03925018692530497, 'EX_phe__L_e': 10.0, 'EX_ca2_e': 0.014549711222763873, 'EX_cys__L_e': 10.0, 'EX_his__L_e': 0.1541538579498835, 'EX_h2o_e': 10.0, 'EX_pi_e': 5.519506278294172, 'EX_cl_e': 0.016447785134853243, 'EX_mn2_e': 0.010614197937497382, 'EX_ni2_e': 0.009935074921301279, 'EX_mg2_e': 0.023991908100634878, 'EX_k_e': 0.47512400263709553, 'EX_ala__L_e': 10.0, 'EX_cu2_e': 0.009176397041292562, 'EX_orn_e': 10.0, 'EX_glc__D_e': 10.0, 'EX_leu__L_e': 0.8034579716308596, 'EX_fe2_e': 0.037761215092602184, 'EX_glu__L_e': 10.0, 'EX_na1_e': 5.090522931794308, 'EX_4abz_e': 0.00018133893906634825, 'EX_lys__L_e': 0.9223436956285925, 'EX_co2_e': 0.13166473758769873, 'EX_pro__L_e': 0.05991910190673909, 'EX_thm_e': 0.

In [10]:
print("Length of media: ", len(media_COMPM_SA))
print(media_COMPM_SA)
save_dict(media_COMPM_SA, "compm_SA.json")

Length of media:  30
{'EX_o2_e': 20.0, 'EX_so4_e': 0.010557173971642781, 'EX_zn2_e': 0.008915031973978057, 'EX_nac_e': 7.953461359822521, 'EX_arg__L_e': 10.0, 'EX_trp__L_e': 0.03925018692530497, 'EX_phe__L_e': 10.0, 'EX_cys__L_e': 10.0, 'EX_his__L_e': 0.1541538579498835, 'EX_h2o_e': 10.0, 'EX_pi_e': 5.519506278294172, 'EX_cl_e': 0.016447785134853243, 'EX_mn2_e': 0.010614197937497382, 'EX_mg2_e': 0.023991908100634878, 'EX_pyr_e': 10.0, 'EX_k_e': 0.47512400263709553, 'EX_orn_e': 10.0, 'EX_glc__D_e': 10.0, 'EX_leu__L_e': 0.8034579716308596, 'EX_urea_e': 10.0, 'EX_fe2_e': 0.037761215092602184, 'EX_glu__L_e': 10.0, 'EX_na1_e': 5.090522931794308, 'EX_ser__L_e': 10.0, 'EX_mobd_e': 1.7910862288293614e-05, 'EX_lys__L_e': 0.9223436956285925, 'EX_co2_e': 0.13166473758769873, 'EX_thm_e': 0.0011411777972255722, 'EX_ribflv_e': 0.0007637848411345633, 'EX_val__L_e': 10.0}


In [11]:
# Check for growth
with model_SA:
    model_SA.medium=media_COMPM_SA
    print("SA growth: ",model_SA.slim_optimize())
with model_DP:
    model_DP.medium=media_COMPM_DP
    print("DP growth: ",model_DP.slim_optimize())

SA growth:  2.5586946126133925
DP growth:  0.2798991966652633


So we have optimal growth for both species, as expected.

In [12]:
# Check that it is valid snm3
for id,val in media_COMPM_DP.items():
    idx = str(id[3:-2])
    assert idx in list(snm3["BiGG"])
for id,val in media_COMPM_SA.items():
    idx = str(id[3:-2])
    assert idx in list(snm3["BiGG"])

In [13]:
print("Length of media: ", len(media_COMPM_DP))
print("Length of media: ", len(media_COMPM_SA))

Length of media:  36
Length of media:  30


## 2) FVA bounds with variable thresholds:

### 2.1) FVA = 0.1

In [14]:
fva_DP_10 = cobra.flux_analysis.flux_variability_analysis(model_DP, fraction_of_optimum=0.1)
fva_SA_10 = cobra.flux_analysis.flux_variability_analysis(model_SA, fraction_of_optimum=0.1)

In [15]:
fva_DP_10.to_excel("fva_DP_010.xlsx")
fva_SA_10.to_excel("fva_SA_010.xlsx")

In [16]:
media_COMPM_DP_10, media_COMPM_SA_10 = compute_COMPM(fva_DP_10, fva_SA_10)
# Check for growth
with model_SA:
    model_SA.medium=media_COMPM_SA_10
    print("SA growth: ",model_SA.slim_optimize())
with model_DP:
    model_DP.medium=media_COMPM_DP_10
    print("DP growth: ",model_DP.slim_optimize())

SA growth:  2.4322218392467847
DP growth:  0.2798991966652633


### 2.2) FVA = 0.4

In [17]:
fva_DP_40 = cobra.flux_analysis.flux_variability_analysis(model_DP, fraction_of_optimum=0.4)
fva_SA_40 = cobra.flux_analysis.flux_variability_analysis(model_SA, fraction_of_optimum=0.4)

In [18]:
fva_DP_40.to_excel("fva_DP_040.xlsx")
fva_SA_40.to_excel("fva_SA_040.xlsx")

In [19]:
media_COMPM_DP_40, media_COMPM_SA_40 = compute_COMPM(fva_DP_40, fva_SA_40)
# Check for growth
with model_SA:
    model_SA.medium=media_COMPM_SA_40
    print("SA growth: ",model_SA.slim_optimize())
with model_DP:
    model_DP.medium=media_COMPM_DP_40
    print("DP growth: ",model_DP.slim_optimize())

SA growth:  2.432143714435963
DP growth:  0.2798991966652633


### 2.3) FVA = 0.8

In [20]:
fva_DP_80 = cobra.flux_analysis.flux_variability_analysis(model_DP, fraction_of_optimum=0.8)
fva_SA_80 = cobra.flux_analysis.flux_variability_analysis(model_SA, fraction_of_optimum=0.8)

In [21]:
fva_DP_80.to_excel("fva_DP_080.xlsx")
fva_SA_80.to_excel("fva_SA_080.xlsx")

In [22]:
media_COMPM_DP_80, media_COMPM_SA_80 = compute_COMPM(fva_DP_80, fva_SA_80)
# Check for growth
with model_SA:
    model_SA.medium=media_COMPM_SA_80
    print("SA growth: ",model_SA.slim_optimize())
with model_DP:
    model_DP.medium=media_COMPM_DP_80
    print("DP growth: ",model_DP.slim_optimize())

SA growth:  2.432221839246782
DP growth:  0.2798991966652633
